# About
 * This notebook uses **LLM agents** to post a job. 

#### a) LLM Agents
   * **seeker**: who gathers information about a position (e.g. `a plant scientist`)
   * **writer**: who writes a job post

    
#### b) Tasks
   * **searching**: the seeker performs industry analysis, collects information on hiring needs, and lists SEO keywords
   * **writing**: the writer uses the information provided and writes a job post

 
#### c) Technical Background
 * **`gemma-7b-it` LLM model** hosted on **Groq**. 
 * **[crewai](https://github.com/joaomdmoura/crewAI)**, a framework to build multi-agent applications.
 * The idea and most of the code were taken from [AI Crew for Jobs postings](https://github.com/joaomdmoura/crewAI-examples/tree/main/job-posting). Adjustments were made to the model and tools.



# 1. Settings

In [1]:
# install packages if needed
#%pip install crewai crewai_tools langchain_groq 

### Import packages

In [2]:
import os


# model
from langchain_groq import ChatGroq

# agent 
from crewai import Agent, Task, Crew

# agent tools
from crewai_tools.tools import EXASearchTool


# display results in Markdown format
from IPython.display import Markdown

# for using GROQ_API and EXA_API_KEY 
# save GROQ_API_KEY and EXA_API_KEY in a file `.env` then load_dotenv()
import dotenv
dotenv.load_dotenv()

True

### Provide input information about a position

In [3]:
company_description = "A Martian agriculture company"
hiring_needs = "A remote-working plant scientist focuses on overcoming potato production barriers on Mars"
specific_benefits = "Your work will have an impact. We offer a competitive salary with high quality health insurance"

# 2. Create agents

### Model

In [4]:
# llm model 
llm_model_id ="gemma-7b-it"

# Use llm model from Groq 
# Get an API here: https://console.groq.com/keys
llm=ChatGroq(temperature=0,
             model_name=llm_model_id)

### Tools

In [5]:
# Initialize the tool for internet searching capabilities
# get an API here: https://dashboard.exa.ai/
exa_search_tool = EXASearchTool()


### Agent 

In [6]:
# define your agents with roles and goals

# role: searching information 
seeker = Agent(
    llm=llm,
    role='Research Analyst',
    goal='Analyze the company description to generate insights on company values and specific needs.',
    tools=[exa_search_tool],
    backstory='Expert in identifying key values and needs from various sources, including websites and brief descriptions.',
    verbose=True,
    allow_delegation=False)

# role: writing
writer = Agent(
    llm=llm,
    role='Job Description Writer',
    goal='Use insights from the Research Analyst to create a detailed, engaging, and enticing job posting.',
    backstory='Skilled in crafting compelling job descriptions that resonate with the company value and attract the right candidates.',
    verbose=True,
    allow_delegation=False)



### Tasks

In [7]:
searching = Task(
    description=(
        f"""
        1. Analyze the provided company description: {company_description}. 
          Conduct an in-depth analysis of the industry. 
          Investigate current trends, challenges, and opportunities within the industry, utilizing market reports, recent developments, and expert opinions.
          Assess how these factors could impact the role being hired for and the overall attractiveness of the position to potential candidates.
		  Consider how the company's position within this industry and its response to these trends could be leveraged to attract top talent. 
		  Compile a report summarizing these insights, specifically how they can be leveraged in a job posting to attract the right candidates.
          Include in your report how the role contributes to addressing industry challenges or seizing opportunities.
        2. Based on the hiring needs: {hiring_needs}, identify the key skills, experiences, and qualities the ideal candidate should possess for the role. 
          Consider the company's competitive landscape and industry trends. 
          Prepare a list of recommended job requirements and qualifications that align with the company's needs and values.
        3. Include SEO keywords"""
    ),
    expected_output="""A comprehensive report detailing the company's values and mission, along with specific selling points relevant to the job role.
    Include a list of recommended skills, experiences, and qualities for the ideal candidate, aligned with the company's values and the specific role's requirements.
    List SEO keywords.
    """,
    agent=seeker,
)

writing = Task(
    description=(
        f"""1. Draft a job posting for the role described by the hiring manager: {hiring_needs}.
          Develop a detailed content outline.
          Use the insights on {company_description} to start with a compelling introduction, 
          followed by a detailed role description, responsibilities, and required skills and qualifications. 
          Ensure the tone aligns with the company's values and incorporate any unique benefits or opportunities offered by the company.
		  Specfic benefits: {specific_benefits}
        2. Review the draft job posting for the role:{hiring_needs}. 
          Check for clarity, engagement, grammatical accuracy, and alignment with the company's values. 
          Edit and refine the content, ensuring it speaks directly to the desired candidates and accurately reflects the role's unique benefits and opportunities. 
        3. Incorporate SEO keywords naturally.     
        4. Sections/Subtitles are properly named in an engaging manner.
        5. Proofread for grammatical errors and add a catchy job post title   
      """
    ),
    expected_output="""
    A detailed, engaging, and polished job posting that includes an introduction, role description, responsibilities, requirements, and unique company benefits.
    The tone should resonate with the company's culture and values, aimed at attracting the right candidates.
    Formated in markdown.""",
    agent=writer,
)


### Assemble the crew and assign tasks

In [8]:
# instantiate your crew with a sequential process
crew = Crew(
  agents=[seeker, writer],
  tasks=[searching, writing],
  verbose=1,
)


### Kick off

In [9]:
# get your crew to work!
result = crew.kickoff()


 [2024-06-22 12:32:57][DEBUG]: == Working Agent: Research Analyst


> Entering new CrewAgentExecutor chain...
## Final Answer

**Thought:** I have analyzed the company description and identified key values, industry trends, and candidate requirements for the remote-working plant scientist role.

**Final Answer:**

**I. Company Values and Industry Analysis:**

The Martian agriculture company prioritizes sustainability, innovation, and collaboration. The industry faces challenges in adapting traditional agricultural practices to Martian conditions, including limited water and soil resources, harsh environmental conditions, and limited infrastructure. Opportunities exist for technological advancements, resource optimization, and sustainable agricultural practices.

**II. Key Skills and Qualities for the Role:**

- **Technical Expertise:** Deep understanding of plant science, agricultural technologies, and Martian environmental conditions.
- **Problem-Solving & Innovation:** Ability to ove

# 3. Result

In [10]:
Markdown(result)

**

## **Plant Scientist - Martian Agriculture Pioneer**

### **Introduction**

Join our revolutionary team of Martian agricultural pioneers at [Company Name]! We're seeking a visionary Plant Scientist to spearhead our mission to overcome potato production barriers on Mars. If you have a deep-seated passion for plant science, a knack for solving complex problems, and a desire to push the boundaries of agricultural innovation, this is your chance to make history.

### **About Our Company**

At [Company Name], we believe in the power of sustainable agriculture to nourish a growing population on Mars. We're driven by a commitment to innovation, collaboration, and environmental responsibility. Our team of experts is dedicated to transforming Martian agriculture through technological advancements, resource optimization, and sustainable practices.

### **Role Overview**

As a Plant Scientist, you will play a pivotal role in developing and implementing cutting-edge technologies to overcome the unique challenges of Martian agriculture. You will work collaboratively with our team to:

- Develop and implement sustainable potato production systems
- Optimize resource utilization through innovative technologies
- Analyze and mitigate environmental factors limiting plant growth
- Monitor and improve soil health and fertility
- Collaborate with engineers and researchers to find creative solutions to production barriers


### **Responsibilities**

- Design and conduct experiments to optimize potato growth and yield
- Develop and implement resource-efficient irrigation and nutrient management strategies
- Analyze soil composition and atmospheric conditions to identify potential limitations
- Evaluate and refine agricultural technologies for Martian conditions
- Communicate research findings and recommendations to stakeholders


### **Required Skills and Qualifications**

- PhD in Plant Science or related field
- 5+ years of experience in agricultural research or development
- Expertise in hydroponics, greenhouse management, and resource optimization
- Familiarity with Martian soil and atmospheric conditions
- Excellent communication and teamwork skills


### **Unique Benefits and Opportunities**

- Competitive salary and benefits package
- Opportunity to work on the forefront of Martian agriculture
- Global connectivity and remote work flexibility
- Be part of a passionate and innovative team
- Make a tangible impact on the future of food production on Mars


**Join us and become a Martian Agriculture Pioneer! Apply today and shape the future of agriculture on the Red Planet!**